# Wpowadzenie do deep learning w bibliotece Flux.jl

## Przykład

Aby w  zrozumieć sposób pracy z Fluxem warto rozpatrzeć prosty przykład. Zajmiemy się przetwarzaniem języka naturalnego - zbudujemy model zdolny do generowania składnej wypowiedzi w języku polskim.

Wyjściowe założenie jest takie, że wytrenujemy sieć neuronową, która będzie estymowała prawdopodobieństwo wystąpienia danego znaku w ciągu na podstawie poprzedzających go znaków w sekwencji ([<b>Character-Level Language Model</b>](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)).

Zbiórem na którym będziemy pracowali jest <i>W poszukiwaniu straconego czasu</i> Marcela Prousta. 

[![](https://upload.wikimedia.org/wikipedia/commons/b/b8/Marcel_Proust_1895.jpg)](https://pl.wikipedia.org/wiki/Marcel_Proust)

>(...) matka widząc, że mi jest zimno, namówiła mnie, abym się napił wbrew zwyczajowi trochę herbaty. Odmówiłem zrazu; potem, nie wiem czemu, namyśliłem się. Posłała po owe krótkie i pulchne ciasteczka zwane magdalenkami, które wyglądają jak odlane w prążkowanej skorupie muszli. I niebawem (...) machinalnie podniosłem do ust łyżeczkę herbaty, w której rozmoczyłem kawałek magdalenki. Ale w tej samej chwili, kiedy łyk pomieszany z okruchami ciasta dotknął mego podniebienia, zadrżałem, czując, że się we mnie dzieje coś niezwykłego. Owładnęła mną rozkoszna słodycz (...). Sprawiła, że w jednej chwili koleje życia stały mi się obojętne, klęski jako błahe, krótkość złudna (...). Cofam się myślą do chwili, w której wypiłem pierwszą łyżeczkę herbaty (...). I nagle wspomnienie zjawiło mi się. Ten smak to była magdalenka cioci Leonii.(...)

Zanim jednak zaczniemy wprowadźmy odrobinę teorii stojącej za tym zagadnieniem:

### Rekurencyjne sieci neuronowe (Recurrent neural networks)

- Charakterystyczną cechą tego typu sieci jest to, że pozwalają one na istnienie wewnątrz grafu cykli skierowanych.
- Oznacza to, że informacja wewnątrz takiej sieci nie musi płynąć tylko w jednym kierunku - neurony leżące na tej samej warstwie także mogą przesyłać sobie wzajemnie dane:

[![](http://karpathy.github.io/assets/rnn/diags.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

Dzięki tej właściwości RNN doskonale nadają się do budowy interesującego nas modelu: 

[![](http://karpathy.github.io/assets/rnn/charseq.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

### Long short-term memory

Problemem na który można natrafić w przypadku korzystania z RNN jest pamięć takiej sieci. Gdy odległość pomiędzy aktualnym a poprzedzającymi go węzłami, które niosą za sobą kluczową informację jest niewielka, sieć jest w stanie efektywnie je wykorzystać:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Problem się pojawia gdy ta odległość jest duża - wtedy kluczowe informacje po prostu znikają w szumie:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Wtedy też, warto zastosować sieć LSTM, która ze względu na swoją architekturę jest w stanie odpowiednio filtrować informację i wykorzystawać je nawet wtedy, gdy ich źródło jest znacznie oddalone od aktualnego neuronu:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Przejdźmy teraz do implementowania modelu za pomocą Fluxa:

### Implementacja

In [1]:
using Flux
using Flux: onehot, argmax, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition

Pierwszym krokiem jest oczywiście odpowiednie przygotowanie danych na których będziemy pracowali:

In [2]:
text = collect(read("w_poszukiwaniu.txt",String));
alphabet = [unique(text)..., '_'];

Następnie kodujemy zmienne jakościowe:

In [3]:
text = map(ch -> onehot(ch, alphabet), text);
stop = onehot('_', alphabet);

In [4]:
N = length(alphabet);
seqlen = 100;
batch_size = 32;

In [5]:
Xs = collect(partition(batchseq(chunk(text, batch_size), stop), seqlen));
Ys = collect(partition(batchseq(chunk(text[2:end], batch_size), stop), seqlen));

In [ ]:
q = chunk("ala ma kota", 3)

In [ ]:
w = batchseq(q, '_')

In [ ]:
for i in partition(w, 2)
   println(i)
end

In [ ]:
collect(partition(w,2))

In [ ]:
collect(partition(batchseq(chunk("ala ma kota"[2:end], 3), '_'),2))

In [8]:
m = Chain(
    LSTM(N, 128),
    Dropout(0.5), 
    LSTM(128, 128),
    Dropout(0.5),
    Dense(128, N),
    softmax)

function loss(xs, ys)
  l = sum(crossentropy.(m.(xs), ys))
  Flux.truncate!(m)
  return l
end

opt = ADAM(0.001)


function sample(m, alphabet, len; temp = 1)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)).data)
  end
  return String(take!(buf))
end

evalcb = () -> @show loss(Xs[5], Ys[5])

#evalcb = function ()
#    @show loss(Xs[5], Ys[5])
#    println(sample(deepcopy(m), alphabet, 100))
#end

#12 (generic function with 1 method)

In [ ]:
Flux.@epochs 4 Flux.train!(loss, params(m), zip(Xs, Ys), opt,
cb = throttle(evalcb, 240))

┌ Info: Epoch 1
└ @ Main C:\Users\p\.julia\packages\Flux\zNlBL\src\optimise\train.jl:105


loss(Xs[5], Ys[5]) = 478.4657f0 (tracked)
loss(Xs[5], Ys[5]) = 332.77264f0 (tracked)
loss(Xs[5], Ys[5]) = 292.60413f0 (tracked)
loss(Xs[5], Ys[5]) = 272.03925f0 (tracked)
loss(Xs[5], Ys[5]) = 261.71442f0 (tracked)
loss(Xs[5], Ys[5]) = 256.8383f0 (tracked)
loss(Xs[5], Ys[5]) = 252.2395f0 (tracked)
loss(Xs[5], Ys[5]) = 248.2264f0 (tracked)
loss(Xs[5], Ys[5]) = 243.43987f0 (tracked)
loss(Xs[5], Ys[5]) = 240.7371f0 (tracked)
loss(Xs[5], Ys[5]) = 237.26335f0 (tracked)
loss(Xs[5], Ys[5]) = 235.19301f0 (tracked)
loss(Xs[5], Ys[5]) = 234.18915f0 (tracked)
loss(Xs[5], Ys[5]) = 233.12976f0 (tracked)


┌ Info: Epoch 2
└ @ Main C:\Users\p\.julia\packages\Flux\zNlBL\src\optimise\train.jl:105


loss(Xs[5], Ys[5]) = 230.86375f0 (tracked)
loss(Xs[5], Ys[5]) = 229.00516f0 (tracked)
loss(Xs[5], Ys[5]) = 228.54037f0 (tracked)
loss(Xs[5], Ys[5]) = 225.23131f0 (tracked)
loss(Xs[5], Ys[5]) = 225.33601f0 (tracked)
loss(Xs[5], Ys[5]) = 224.24614f0 (tracked)
loss(Xs[5], Ys[5]) = 221.65182f0 (tracked)
loss(Xs[5], Ys[5]) = 220.89214f0 (tracked)
loss(Xs[5], Ys[5]) = 219.32072f0 (tracked)
loss(Xs[5], Ys[5]) = 216.43709f0 (tracked)
loss(Xs[5], Ys[5]) = 216.82336f0 (tracked)
loss(Xs[5], Ys[5]) = 215.46017f0 (tracked)
loss(Xs[5], Ys[5]) = 214.70876f0 (tracked)


┌ Info: Epoch 3
└ @ Main C:\Users\p\.julia\packages\Flux\zNlBL\src\optimise\train.jl:105


loss(Xs[5], Ys[5]) = 213.86687f0 (tracked)
loss(Xs[5], Ys[5]) = 211.24521f0 (tracked)
loss(Xs[5], Ys[5]) = 209.4972f0 (tracked)
loss(Xs[5], Ys[5]) = 208.56644f0 (tracked)
loss(Xs[5], Ys[5]) = 209.90005f0 (tracked)
loss(Xs[5], Ys[5]) = 206.6048f0 (tracked)
loss(Xs[5], Ys[5]) = 206.75063f0 (tracked)
loss(Xs[5], Ys[5]) = 206.10915f0 (tracked)


┌ Info: Epoch 4
└ @ Main C:\Users\p\.julia\packages\Flux\zNlBL\src\optimise\train.jl:105


loss(Xs[5], Ys[5]) = 204.05675f0 (tracked)
loss(Xs[5], Ys[5]) = 203.73071f0 (tracked)
loss(Xs[5], Ys[5]) = 203.47665f0 (tracked)
loss(Xs[5], Ys[5]) = 201.3106f0 (tracked)
loss(Xs[5], Ys[5]) = 201.97388f0 (tracked)
loss(Xs[5], Ys[5]) = 200.84041f0 (tracked)
loss(Xs[5], Ys[5]) = 201.17735f0 (tracked)
loss(Xs[5], Ys[5]) = 199.21379f0 (tracked)
loss(Xs[5], Ys[5]) = 199.85109f0 (tracked)
loss(Xs[5], Ys[5]) = 200.21931f0 (tracked)
loss(Xs[5], Ys[5]) = 201.95644f0 (tracked)
